### Dataset Download link : https://archive.ics.uci.edu/ml/machine-learning-databases/00352/

In [3]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_excel(r'C:/Users/sandeep/Downloads\Online Retail.xlsx')

In [5]:
df = df.head(1000).copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df.count()
# remove missing values
df.dropna(inplace=True)
# again check missing values
df.isnull().sum()
df.count()

InvoiceNo      999
StockCode      999
Description    999
Quantity       999
InvoiceDate    999
UnitPrice      999
CustomerID     999
Country        999
dtype: int64

Given below is the description of the fields in this dataset:

InvoiceNo: Invoice number, a unique number assigned to each transaction.

StockCode: Product/item code. a unique number assigned to each distinct product.

Description: Product description

Quantity: The quantities of each product per transaction.

InvoiceDate: Invoice Date and time. The day and time when each transaction was generated.

CustomerID: Customer number, a unique number assigned to each customer.

In [8]:
df['StockCode']= df['StockCode'].astype(str)

In [9]:
customers = df["CustomerID"].unique().tolist()
len(customers)

48

In [10]:
"""
There are 48 customers in our dataset. For each of these customers we will extract their buying history. 
In other words, we can have 48 sequences of purchases.

It is a good practice to set aside a small part of the dataset for validation purpose. 
Therefore, I will use data of 90% of the customers to create word2vec embeddings. Let's split the data.
"""


random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]

In [11]:
"""
Let's create sequences of purchases made by the customers in the dataset for both the train and validation set.
"""


# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_train.append(temp)
    

# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['CustomerID'].unique()):
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 777.36it/s]


## Build word2vec Embeddings for Products

In [16]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(purchases_train, progress_per=200)

model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(311, 8660)

In [17]:
# save word2vec model
model.save("word2vec_2.model")

"""
As we do not plan to train the model any further, we are calling init_sims(), 
which will make the model much more memory-efficient.
"""

model.init_sims(replace=True)

"""
Now we will extract the vectors of all the words in our vocabulary and store it in one place for easy access.
"""

# extract all vectors
X = model[model.wv.vocab]

X.shape


(27, 100)

## Visualize word2vec Embeddings
It is always quite helpful to visualize the embeddings that you have created. Over here we have 100 dimensional embeddings. We can't even visualize 4 dimensions let alone 100. Therefore, we are going to reduce the dimensions of the product embeddings from 100 to 2 by using the UMAP algorithm, it is used for dimensionality reduction.

In [31]:
import umap

"""
Every dot in this plot is a product. As you can see, there are several tiny clusters of these datapoints. 
These are groups of similar products.
"""

cluster_embedding = umap.UMAP(n_neighbors=30, min_dist=0.0,
                              n_components=2, random_state=42).fit_transform(X)

plt.figure(figsize=(10,9))
plt.scatter(cluster_embedding[:, 0], cluster_embedding[:, 1], s=3, cmap='Spectral')

ModuleNotFoundError: No module named 'umap'

### Start Recommending Products
We are finally ready with the word2vec embeddings for every product in our online retail dataset. Now our next step is to suggest similar products for a certain product or a product's vector.

Let's first create a product-ID and product-description dictionary to easily map a product's description to its ID and vice versa.

In [20]:

products = train_df[["StockCode", "Description"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

In [21]:
# test the dictionary
products_dict['84029E']

['RED WOOLLY HOTTIE WHITE HEART.']

In [22]:
"""
I have defined the function below. It will take a product's vector (n) as input and return top 6 similar products.
"""  
def similar_products(v, n = 6):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms


In [24]:
"""
Let's try out our function by passing the vector of the product '90019A' ('SILVER M.O.P ORBIT BRACELET')
"""
similar_products(model['84029E'])

[('KNITTED UNION FLAG HOT WATER BOTTLE', 0.9375370144844055),
 ('GLASS STAR FROSTED T-LIGHT HOLDER', 0.9371752738952637),
 ('HAND WARMER RED RETROSPOT', 0.9325289726257324),
 ('CREAM CUPID HEARTS COAT HANGER', 0.927280068397522),
 ('WHITE HANGING HEART T-LIGHT HOLDER', 0.9178310036659241),
 ('SET 7 BABUSHKA NESTING BOXES', 0.9102854132652283)]

The results are pretty relevant and match well with the input product. However, this output is based on the vector of a single product only. What if we want recommend a user products based on the multiple purchases he or she has made in the past?

One simple solution is to take average of all the vectors of the products he has bought so far and use this resultant vector to find similar products. For that we will use the function below that takes in a list of product ID's and gives out a 100 dimensional vector which is mean of vectors of the products in the input list.

In [58]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [59]:
#If you can recall, we have already created a separate list of purchase sequences for validation purpose. Now let's make use of that.

print(len(purchases_val[1]))

#The length of the first list of products purchased by a user is 314. We will pass this products' sequence of the validation set to the function aggregate_vectors.

aggregate_vectors(purchases_val[1])

12


nan


Well, the function has returned an array of 100 dimension. It means the function is working fine. Now we can use this result to get the most similar products. Let's do it.

In [60]:
similar_products(aggregate_vectors(purchases_val[1]))

TypeError: 'numpy.float64' object is not iterable

In [30]:
"""
As it turns out, our system has recommended 6 products based on the entire purchase history of a user. Moreover,
if you want to get products suggestions based on the last few purchases only then also you can use the same set of functions.

Below I am giving only the last 10 products purchased as input.
"""
similar_products(aggregate_vectors(purchases_val[0][-10:]))

TypeError: 'numpy.float64' object is not iterable

Reference : https://www.analyticsvidhya.com/blog/2019/07/how-to-build-recommendation-system-word2vec-python/?utm_source=blog&utm_medium=graph-feature-extraction-deepwalk